In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import radians, cos, sin, asin, sqrt
from sklearn.svm import OneClassSVM
#from functionsPredictions import *
from sklearn import preprocessing
import matplotlib.pyplot as plt
import category_encoders as ce
from datetime import timedelta
from scipy import stats
import seaborn as sns
import xgboost as xgb
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import glob

import lightgbm as lgb


# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_holiday2)

In [35]:
def add_time_vars(data, onehot=True):
    '''
    Adds columns for the month and weekday, and also the one-hot encoding or the cyclical versions of those features.

    :data: Dataframe that contains the a column with the datetime
    :onehot: Use onehot encoding if true and cyclical features if false (default = True)
    
    Returns a Dataframe with either the one-hot encoding or the sine and cosine of the month, weekday and time added
    '''
    data = data.reset_index()
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    if onehot == True:
        data['Year'] = pd.Categorical(data['Datetime'].dt.year)
        data['Month'] = pd.Categorical(data['Datetime'].dt.month)
        data['Weekday'] = pd.Categorical(data['Datetime'].dt.weekday)
        data['Hour'] =  pd.Categorical(data['Datetime'].dt.hour)
        data['Minute'] =  pd.Categorical(data['Datetime'].dt.minute)

        year_dummies = pd.get_dummies(data[['Year']], prefix='Year_')
        month_dummies = pd.get_dummies(data[['Month']], prefix='Month_')
        weekday_dummies = pd.get_dummies(data[['Weekday']], prefix='Weekday_')
        hour_dummies = pd.get_dummies(data[['Hour']], prefix='Hour_')
        minute_dummies = pd.get_dummies(data[['Minute']], prefix='Minute_')
        
        data = data.merge(year_dummies, left_index = True, right_index = True)
        data = data.merge(month_dummies, left_index = True, right_index = True)
        data = data.merge(weekday_dummies, left_index = True, right_index = True)
        data = data.merge(hour_dummies, left_index = True, right_index = True)
        data = data.merge(minute_dummies, left_index = True, right_index = True)
        
    data = data.set_index('Datetime')
    return data


In [40]:
df_lgbm = pd.read_csv("Merged_data.csv")
df_lgbm['Visits'] = df_lgbm['Visits'].astype('int')

In [41]:
df_lgbm = df_lgbm[df_lgbm.Location == 'Erasmuspark']
df_lgbm = add_time_vars(df_lgbm)

In [42]:
X = df_lgbm.drop(columns='Visits')
y = df_lgbm['Visits']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [43]:
a = X_train.columns.tolist()[4:]

X_train = X_train[a]
X_test = X_test[a]

In [ ]:
model = lgb.LGBMClassifier(learning_rate=0.09,max_depth=5,random_state=42)
model.fit(X_train,y_train)

In [19]:
# predict the results
y_pred=clf.predict(X_test)

In [30]:
X_test.iloc[:,10:]

,Thunder,Ice,Holiday_Count,Year,Month,Day,Holiday_Name_Ascension Thursday,Holiday_Name_Easter,Holiday_Name_Fall holiday,Holiday_Name_Good Friday,Holiday_Name_Kings day,Holiday_Name_Liberation Day,Holiday_Name_May holiday,Holiday_Name_Spring holiday,Holiday_Name_Summer holiday,Holiday_Name_Whit,Holiday_Name_Winter holiday
319335,0,0,0.0,2021,6,9,0,0,0,0,0,0,0,0,0,0,0
409015,0,0,1.0,2021,7,26,0,0,0,0,0,0,0,0,1,0,0
38675,0,0,0.0,2020,11,25,0,0,0,0,0,0,0,0,0,0,0
467675,0,0,1.0,2021,8,25,0,0,0,0,0,0,0,0,1,0,0
581495,0,0,1.0,2021,10,24,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183533,0,0,0.0,2021,3,13,0,0,0,0,0,0,0,0,0,0,0
126469,0,0,0.0,2021,1,30,0,0,0,0,0,0,0,0,0,0,0
110635,0,0,0.0,2021,1,18,0,0,0,0,0,0,0,0,0,0,0
156107,0,0,1.0,2021,2,21,0,0,0,0,0,0,0,1,0,0,0


In [15]:
X_test

,Journeys,Windspeed,Temperature,Clouds,Rain amount,Rain duration,Sun duration,Fog,Rain,Snow,...,Holiday_Name_Easter,Holiday_Name_Fall holiday,Holiday_Name_Good Friday,Holiday_Name_Kings day,Holiday_Name_Liberation Day,Holiday_Name_May holiday,Holiday_Name_Spring holiday,Holiday_Name_Summer holiday,Holiday_Name_Whit,Holiday_Name_Winter holiday
319335,499.00,6.00,22.800,1.0,0.0000,0.0,13.5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
409015,0.00,1.00,18.600,8.0,0.0000,0.0,4.5,0,0,0,...,0,0,0,0,0,0,0,1,0,0
38675,231.50,3.25,8.525,8.0,0.0125,0.0,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
467675,401.50,5.00,15.875,8.0,0.0125,0.0,0.0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
581495,0.00,4.00,7.400,8.0,0.0000,0.0,0.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183533,494.00,14.00,6.900,8.0,0.0500,3.0,3.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
126469,0.00,6.00,0.900,7.0,0.0000,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110635,142.25,6.00,5.925,8.0,0.0000,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
156107,0.00,4.00,9.275,8.0,0.0000,0.0,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
